In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.individual_trial_analysis_utils import SessionData, get_photometry_around_event, get_next_centre_poke, get_peak_each_trial_psychometric, get_next_reward_time, HeatMapParams

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
class CustomAlignedDataRewardBlocks(object):
    def __init__(self, session_data, params, reward_block):
        saving_folder = 'W:\\photometry_2AC\\processed_data\\' + session_data.mouse + '\\'
        restructured_data_filename = session_data.mouse + '_' + session_data.date + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = session_data.mouse + '_' + session_data.date + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)

        fiber_options = np.array(['left', 'right'])
        fiber_side_numeric = (np.where(fiber_options == session_data.fiber_side)[0] + 1)[0]
        contra_fiber_side_numeric = (np.where(fiber_options != session_data.fiber_side)[0] + 1)[0]

        self.contra_data = ZScoredTracesRewardBlocks(trial_data, dff,params, contra_fiber_side_numeric, contra_fiber_side_numeric,reward_block)
        self.contra_data.get_peaks()

In [7]:
def get_all_experimental_records():
    experiment_record = pd.read_csv('W:\\photometry_2AC\\experimental_record.csv')
    experiment_record['date'] = experiment_record['date'].astype(str)
    return experiment_record

In [8]:
def open_experiment(experiment_to_add):
    for index, experiment in experiment_to_add.iterrows():
        saving_folder = 'W:\\photometry_2AC\\processed_data\\' + experiment['mouse_id'] + '\\'
        restructured_data_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'restructured_data.pkl'
        trial_data = pd.read_pickle(saving_folder + restructured_data_filename)
        dff_trace_filename = experiment['mouse_id'] + '_' + experiment['date'] + '_' + 'smoothed_signal.npy'
        dff = np.load(saving_folder + dff_trace_filename)
        session_traces = SessionData(experiment['fiber_side'], experiment['recording_site'], experiment['mouse_id'], experiment['date'])
    return trial_data, session_traces

In [9]:
class ZScoredTracesRewardBlocks(object):
    def __init__(self,  trial_data, dff, params, response, first_choice, reward_block):
        self.trial_peaks = None
        self.params = HeatMapParams(params, response, first_choice)
        self.reward_block = reward_block
        self.time_points, self.mean_trace, self.sorted_traces, self.reaction_times, self.state_name, title, self.sorted_next_poke, self.trial_nums, self.event_times, self.outcome_times = find_and_z_score_traces_blocks(
            trial_data, dff, self.params, self.reward_block)

    def get_peaks(self):
        self.trial_peaks = get_peak_each_trial_psychometric(self.sorted_traces, self.time_points, self.outcome_times)

In [10]:
def find_and_z_score_traces_blocks(trial_data, demod_signal, params, reward_block, norm_window=8, sort=False, get_photometry_data=True):
    response_names = ['both left and right', 'left', 'right']
    outcome_names = ['incorrect', 'correct', 'both correct and incorrect']
    events_of_int = trial_data.loc[(trial_data['State type'] == params.state)]
    if  params.state == 10:
        omission_events = trial_data.loc[(trial_data['State type'] == params.state)]
        trials_of_int = omission_events['Trial num'].values
        omission_trials_all_states = trial_data.loc[(trial_data['Trial num'].isin(trials_of_int))]
        events_of_int = omission_trials_all_states.loc[(omission_trials_all_states['State type'] == 4)]
    else:
        events_of_int = trial_data.loc[(trial_data['State type'] == params.state)]
    if params.response != 0:
        events_of_int = events_of_int.loc[events_of_int['Response'] == params.response]
    if params.first_choice != 0:
        events_of_int = events_of_int.loc[events_of_int['First response'] == params.first_choice]
    if params.last_response != 0:
        events_of_int = events_of_int.loc[events_of_int['Last response'] == params.last_response]
        title = ' last response: ' + response_names[params.last_response]
    else:
        title = response_names[params.response]
    if not params.outcome == 2: # if you don't care about the reward or not
        events_of_int = events_of_int.loc[events_of_int['Trial outcome'] == params.outcome]
    #events_of_int = events_of_int.loc[events_of_int['Last outcome'] == 0]

    if params.cue == 'high':
        events_of_int = events_of_int.loc[events_of_int['Trial type'] == 7]
    elif params.cue == 'low':
        events_of_int = events_of_int.loc[events_of_int['Trial type'] == 1]
    events_of_int = events_of_int.loc[events_of_int['Reward block'] == reward_block]



    if params.state == 10:
        title = title + ' ' + 'omission'
    else:
        title = title + ' ' + outcome_names[params.outcome]

    if params.instance == -1:
        events_of_int = events_of_int.loc[
            (events_of_int['Instance in state'] / events_of_int['Max times in state'] == 1)]
    elif params.instance == 1:
        events_of_int = events_of_int.loc[(events_of_int['Instance in state'] == 1)]
        if params.no_repeats == 1:
            events_of_int = events_of_int.loc[events_of_int['Max times in state'] == 1]
    elif params.instance == 0:
        events_of_int = events_of_int

    if params.first_choice_correct != 0:
        events_of_int = events_of_int.loc[
            (events_of_int['First choice correct'] == 1)]

    event_times = events_of_int[params.align_to].values
    trial_nums = events_of_int['Trial num'].values
    state_name = events_of_int['State name'].values[0]
    other_event = np.asarray(
        np.squeeze(events_of_int[params.other_time_point].values) - np.squeeze(events_of_int[params.align_to].values))
    next_centre_poke = get_next_centre_poke(trial_data, events_of_int)
    outcome_times = get_next_reward_time(trial_data, events_of_int)
    outcome_times = outcome_times - event_times

    last_trial_num = events_of_int['Trial num'].unique()[-1]
    events_reset_indx = events_of_int.reset_index(drop=True)
    last_trial_event_indx = events_reset_indx.loc[(events_reset_indx['Trial num'] == last_trial_num)].index
    next_centre_poke[last_trial_event_indx] = events_reset_indx[params.align_to].values[last_trial_event_indx]
    next_centre_poke_norm = next_centre_poke - event_times

    # this all deals with getting photometry data
    if get_photometry_data == True:
        event_photo_traces = get_photometry_around_event(event_times, demod_signal, pre_window=norm_window,
                                                         post_window=norm_window)
        norm_traces = stats.zscore(event_photo_traces.T, axis=0)

        if len(other_event) != norm_traces.shape[1]:
            other_event = other_event[:norm_traces.shape[1]]
        if sort:
            arr1inds = other_event.argsort()
            sorted_other_event = other_event[arr1inds[::-1]]
            sorted_traces = norm_traces.T[arr1inds[::-1]]
            sorted_next_poke = next_centre_poke_norm[arr1inds[::-1]]
        else:
            sorted_other_event = other_event
            sorted_traces = norm_traces.T
            sorted_next_poke = next_centre_poke_norm

        time_points = np.linspace(-norm_window, norm_window, norm_traces.shape[0], endpoint=True, retstep=False, dtype=None,
                             axis=0)
        mean_trace = np.mean(sorted_traces, axis=0)

        return time_points, mean_trace, sorted_traces, sorted_other_event, state_name, title, sorted_next_poke, trial_nums, event_times, outcome_times
    else:
        if sort:
            arr1inds = other_event.argsort()
            sorted_other_event = other_event[arr1inds[::-1]]
            sorted_next_poke = next_centre_poke_norm[arr1inds[::-1]]
        else:
            sorted_other_event = other_event
            sorted_next_poke = next_centre_poke_norm
        return sorted_other_event, state_name, title, sorted_next_poke, trial_nums, event_times




In [11]:
mouse_id = 'SNL_photo22'
date = '20200917'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
behavioural_data, session_data = open_experiment(experiment_to_process)

In [12]:
behavioural_data

,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
0,0.0,1.0,1.0,1.0,1.0,TrialStart,22.6082,22.6182,1.0,-1.0,-1.0,22.6082,28.1883,1.0,1.0,1.0,1.0
1,0.0,1.0,2.0,1.0,2.0,WaitForPoke,22.6182,27.1102,1.0,-1.0,-1.0,22.6082,28.1883,1.0,1.0,1.0,1.0
2,0.0,1.0,3.0,1.0,2.0,CueDelay,27.1102,27.1495,1.0,-1.0,-1.0,22.6082,28.1883,1.0,1.0,1.0,1.0
3,0.0,1.0,2.0,2.0,2.0,WaitForPoke,27.1495,27.3240,1.0,-1.0,-1.0,22.6082,28.1883,1.0,1.0,1.0,1.0
4,0.0,1.0,3.0,2.0,2.0,CueDelay,27.3240,27.4240,1.0,-1.0,-1.0,22.6082,28.1883,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3250,351.0,7.0,3.0,1.0,1.0,CueDelay,2950.8771,2950.9771,2.0,1.0,0.0,2832.8824,2951.6998,1.0,2.0,2.0,1.0
3251,351.0,7.0,4.0,1.0,1.0,WaitForPortOut,2950.9771,2951.0676,2.0,1.0,0.0,2832.8824,2951.6998,1.0,2.0,2.0,1.0
3252,351.0,7.0,5.0,1.0,1.0,WaitForResponse,2951.0676,2951.6883,2.0,1.0,0.0,2832.8824,2951.6998,1.0,2.0,2.0,1.0
3253,351.0,7.0,7.0,1.0,1.0,RightReward,2951.6883,2951.6997,2.0,1.0,0.0,2832.8824,2951.6998,1.0,2.0,2.0,1.0


In [14]:
all_reward_block_data = []
for reward_block in range(1,6):
    print(reward_block)
    try:
        params = {'state_type_of_interest': 5,
            'outcome': 2,
            'last_outcome': 0,  # NOT USED CURRENTLY
            'no_repeats' : 0,
            'last_response': 0,
            'align_to' : 'Time start',
            'instance':- 1,
            'plot_range': [-6, 6],
            'first_choice_correct': 0,
             'cue': 'None'}
        all_reward_block_data.append(CustomAlignedDataRewardBlocks(session_data, params, reward_block))
    except:
        pass
    

1
(51, 160000)
2
(34, 160000)
3
(23, 160000)
4
(53, 160000)
5
(20, 160000)


In [70]:
colours = cm.inferno(np.linspace(0, 1, 5))
fig, ax = plt.subplots(1,1)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        mean_trace = decimate(one_side_data.mean_trace, 10)
        time_points = decimate(one_side_data.time_points, 10)
        traces = decimate(one_side_data.sorted_traces, 10)
        ax.plot(time_points, mean_trace, lw=1.5, color=colours[trial_type-1], label= trial_type)
        ax.axvline(0, color='k')
    except:
        pass
plt.legend()


In [24]:
trial_nums = all_reward_block_data[3].contra_data.trial_nums
trial_before_nums = all_reward_block_data[3].contra_data.trial_nums -1
trials = behavioural_data.loc[behavioural_data['Trial num'].isin(trial_nums)]
trials_before = behavioural_data.loc[behavioural_data['Trial num'].isin(trial_before_nums)]

In [105]:
block_types = pd.DataFrame({'block type': [1, 2, 3, 4, 5], 'left reward': [6, 4, 2, 2, 2], 'right reward': [2, 2, 2, 4, 6]})

In [106]:
block_types

,block type,left reward,right reward
0,1,6,2
1,2,4,2
2,3,2,2
3,4,2,4
4,5,2,6


In [110]:
block_changes

,index,Trial num,Trial type,State type,Instance in state,Max times in state,State name,Time start,Time end,Response,Last response,Last outcome,Trial start,Trial end,Trial outcome,First response,Reward block,First choice correct
0,218,19.0,7.0,1.0,1.0,1.0,TrialStart,186.8660,186.8760,2.0,2.0,1.0,186.8660,189.8826,1.0,2.0,4.0,1.0
1,449,39.0,1.0,1.0,1.0,1.0,TrialStart,292.9503,292.9603,1.0,2.0,1.0,292.9503,300.4817,1.0,1.0,1.0,1.0
2,679,59.0,1.0,1.0,1.0,1.0,TrialStart,409.2106,409.2206,1.0,1.0,1.0,409.2106,412.8748,1.0,1.0,5.0,1.0
3,880,79.0,7.0,1.0,1.0,1.0,TrialStart,529.2831,529.2931,2.0,2.0,1.0,529.2831,535.2059,1.0,2.0,3.0,1.0
4,1061,99.0,7.0,1.0,1.0,1.0,TrialStart,603.4020,603.4120,2.0,1.0,1.0,603.4020,606.3605,1.0,2.0,1.0,1.0
5,1248,119.0,7.0,1.0,1.0,1.0,TrialStart,758.1454,758.1554,2.0,2.0,1.0,758.1454,761.1484,1.0,2.0,4.0,1.0
6,1426,139.0,7.0,1.0,1.0,1.0,TrialStart,870.5293,870.5393,2.0,2.0,1.0,870.5293,874.9733,1.0,2.0,2.0,1.0
7,1601,159.0,7.0,1.0,1.0,1.0,TrialStart,942.6276,942.6376,2.0,1.0,1.0,942.6276,946.0595,1.0,2.0,1.0,1.0
8,1776,179.0,1.0,1.0,1.0,1.0,TrialStart,1060.7022,1060.7122,1.0,1.0,1.0,1060.7022,1063.2753,1.0,1.0,2.0,1.0
9,1950,199.0,7.0,1.0,1.0,1.0,TrialStart,1202.5151,1202.5251,1.0,1.0,1.0,1202.5151,1208.5605,0.0,1.0,5.0,0.0


In [166]:
first_contra_trial_post_block_change = {}
first_contra_trial = []
current_reward_amounts = []
change_in_reward_amounts = []
relative_values = []
change_in_relative_values = []
contra_side = 2
block_diff_value = behavioural_data['Reward block'].diff()
block_changes = behavioural_data[block_diff_value!= 0]
block_changes = block_changes.reset_index()
previous_blocks = block_changes['Reward block']
block_changes = block_changes.drop(block_changes.index[0])
previous_blocks = previous_blocks.drop(previous_blocks.index[-1])
block_changes = block_changes.reset_index()
first_contra_trial_post_block_change['previous block type'] = previous_blocks
first_contra_trial_post_block_change['new block type'] = block_changes['Reward block']
for index, block_change in block_changes.iterrows():
    trial_num = block_change['Trial num']
    counter = 0
    current_trial = block_change
    current_response = current_trial['Response']
    current_trial_num = current_trial['Trial num']
    while current_response != contra_side:
        counter += 1
        current_trial = behavioural_data[(behavioural_data['Trial num'] == trial_num + counter)]
        current_trial = current_trial[(current_trial['State name'] == 'TrialStart')]
        current_response = current_trial['Response'].values
        current_trial_num = current_trial['Trial num'].values[0]
    
    if contra_side == 1:
        new_reward_amount = block_types[block_types['block type'] == block_change['Reward block']] ['right reward'].values[0]
        new_other_amount =  block_types[block_types['block type'] == block_change['Reward block']] ['left reward'].values[0]
        old_reward_amount = block_types[block_types['block type'] == previous_blocks[index]['Reward block']] ['right reward'].values[0]
        old_other_amount = block_types[block_types['block type'] == previous_blocks[index]['Reward block']] ['left reward'].values[0]
    else:
        new_reward_amount = block_types[block_types['block type'] == block_change['Reward block']] ['left reward'].values[0]
        new_other_amount =  block_types[block_types['block type'] == block_change['Reward block']] ['right reward'].values[0]
        old_reward_amount = block_types[block_types['block type'] == previous_blocks[index]] ['left reward'].values[0]
        old_other_amount = block_types[block_types['block type'] == previous_blocks[index]] ['right reward'].values[0]
    current_reward_amounts.append(new_reward_amount)
    change_in_reward_amounts.append(new_reward_amount -old_reward_amount)
    relative_values.append(new_reward_amount - new_other_amount)
    change_in_relative_values.append((new_reward_amount - new_other_amount) - (old_reward_amount - old_other_amount))
    first_contra_trial.append(current_trial_num)

first_contra_trial_post_block_change['first contra trial num'] = first_contra_trial
first_contra_trial_post_block_change['block change trial num'] = block_changes['Trial num']
first_contra_trial_post_block_change['current reward amounts'] = current_reward_amounts
first_contra_trial_post_block_change['change in reward amounts'] = change_in_reward_amounts
first_contra_trial_post_block_change['relative values'] = relative_values
first_contra_trial_post_block_change['change in relative value'] = change_in_relative_values



    # previous block
    # current block
    # change in reward amount (ul)
    # change in relative value

In [167]:
pd.DataFrame(first_contra_trial_post_block_change)

,previous block type,new block type,first contra trial num,block change trial num,current reward amounts,change in reward amounts,relative values,change in relative value
0,1.0,4.0,19.0,19.0,2,-4,-2,-6
1,4.0,1.0,40.0,39.0,6,4,4,6
2,1.0,5.0,60.0,59.0,2,-4,-4,-8
3,5.0,3.0,79.0,79.0,2,0,0,4
4,3.0,1.0,99.0,99.0,6,4,4,4
5,1.0,4.0,119.0,119.0,2,-4,-2,-6
6,4.0,2.0,139.0,139.0,4,2,2,4
7,2.0,1.0,159.0,159.0,6,2,4,2
8,1.0,2.0,182.0,179.0,4,-2,2,-2
9,2.0,5.0,203.0,199.0,2,-2,-4,-6


In [162]:
change_in_relative_values

[-6, 6, -8, 4, 4, -6, 4, 2, -2, -6, 2, 6, -6, 2, -2, 6, -2]

In [108]:
index

3153

In [80]:
block_diff_value

0       NaN
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
3250    0.0
3251    0.0
3252    0.0
3253    0.0
3254    0.0
Name: Reward block, Length: 3255, dtype: float64

In [75]:
colours = cm.inferno(np.linspace(0,1, 5))
fig2, ax = plt.subplots(1,3)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        x_vals = np.ones([len(one_side_data.trial_peaks)])*trial_type
        ax[0].hist(one_side_data.trial_peaks, color=colours[trial_type-1], alpha=0.4)
        ax[1].scatter(trial_type, np.mean(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[1].set_ylabel('Mean peak')
        ax[2].scatter(trial_type, np.median(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[2].set_ylabel('Median peak')
        print(np.mean(one_side_data.trial_peaks), np.median(one_side_data.trial_peaks))
    except:
        pass

0.8719974503007026 0.8838530247845447
0.734418515406605 0.5818760037034908
1.3165566302983154 1.1614116229862628
0.7800667694836998 0.663554277871089
0.8304953025800229 0.5631001717818181


In [217]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

0.15315428214827714

In [ ]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

In [209]:
np.asarray(one_side_data.trial_peaks)/4

array([0.14386391, 0.36703822, 0.03294286, 0.67309225, 0.1799292 ,
       0.26867563, 0.06190406])

In [199]:
np.median([0.13636681, 0.46637622, 0.51340394])

0.46637622

In [207]:
np.median(np.concatenate([np.asarray(one_side_data.trial_peaks),np.asarray([0.13636681, 0.46637622, 0.51340394])]))

0.5444297804051131